In [13]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.special import softmax

In [14]:
bc = datasets.load_iris()
X, y = bc.data, bc.target

In [15]:
def one_hot(y, num_classes):
    return np.eye(num_classes)[y]

In [17]:
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

In [18]:
def cross_entropy_loss(y_true, y_pred):
    n_samples = y_true.shape[0]
    return -np.sum(y_true * np.log(y_pred + 1e-8)) / n_samples

In [19]:
def predict_proba(X, weights, bias):
    linear_model = np.dot(X, weights) + bias
    return softmax(linear_model)

In [20]:
def predict(X, weights, bias):
    y_pred = predict_proba(X, weights, bias)
    return np.argmax(y_pred, axis=1)

In [21]:
class LogisticRegression:
    def __init__(self, lr=0.01, num_epochs=1000, num_classes=3):
        self.lr = lr
        self.num_epochs = num_epochs
        self.num_classes = num_classes
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        y_one_hot = one_hot(y, self.num_classes)

        self.weights = np.zeros((n_features, self.num_classes))
        self.bias = np.zeros((1, self.num_classes))

        for epoch in range(self.num_epochs):
            y_pred = predict_proba(X, self.weights, self.bias)
            loss = cross_entropy_loss(y_one_hot, y_pred)

            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y_one_hot))
            db = (1 / n_samples) * np.sum(y_pred - y_one_hot, axis=0)
            
            self.weights -= self.lr * dw
            self.bias -= self.lr * db
            
            if (epoch + 1) % 100 == 0:
                print(f'Epoch: {epoch+1}, Loss: {loss:.4f}')

    def evaluate(self, X, y):
        y_pred = predict(X, self.weights, self.bias)
        accuracy = np.mean(y_pred == y)
        return accuracy

In [22]:
sc = StandardScaler()
X = sc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

model = LogisticRegression(lr=0.1, num_epochs=1000, num_classes=3)

model.fit(X_train, y_train)

accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

Epoch: 100, Loss: 0.3323
Epoch: 200, Loss: 0.2676
Epoch: 300, Loss: 0.2295
Epoch: 400, Loss: 0.2036
Epoch: 500, Loss: 0.1847
Epoch: 600, Loss: 0.1703
Epoch: 700, Loss: 0.1590
Epoch: 800, Loss: 0.1498
Epoch: 900, Loss: 0.1422
Epoch: 1000, Loss: 0.1358
Accuracy: 100.00%
